In [1]:
import os
import pathlib

from tqdm.auto import tqdm

In [2]:
from facade_project import FACADE_LABELME_ORIGINAL_DIR, FACADE_IMAGES_DIR, LABEL_NAME_TO_VALUE, NUM_IMAGES

In [3]:
img_paths = [os.path.join(FACADE_LABELME_ORIGINAL_DIR, fname) for fname in sorted(os.listdir(FACADE_LABELME_ORIGINAL_DIR))]

len(img_paths)

418

# From Labelme to COCO

In [4]:
from facade_project.coco.from_labelme import Labelme2coco

In [5]:
def create_coco_ann_and_images_dir(json_files, dir_name, name_suffix, selected_classes=LABEL_NAME_TO_VALUE):
    data_dir = '{}/coco/{}'.format(FACADE_IMAGES_DIR, dir_name)
    img_dir = '{}/images/{}'.format(data_dir, name_suffix)
    pathlib.Path(img_dir).mkdir(parents=True, exist_ok=True)
    l2c_obj = Labelme2coco(
        json_files,
        '{}/ann_{}.json'.format(data_dir, name_suffix),
        only_labels=selected_classes,
        save_img_dir=img_dir,
    )
    l2c_obj.save_json()
    return l2c_obj

# Split train - val

In [6]:
from facade_project.data import get_indices_split

In [7]:
train_ind, val_ind = get_indices_split(NUM_IMAGES)

In [8]:
train_json = [img_paths[i] for i in train_ind]
val_json = [img_paths[i] for i in val_ind]

# Generate
Run this only to generate the coco dataset again (careful about overwriting old ones)

## Training

In [ ]:
l2c_obj = create_coco_ann_and_images_dir(
    json_files=train_json,
    dir_name='original',
    name_suffix='train',
)

## Validation

In [ ]:
create_coco_ann_and_images_dir(
    json_files=val_json,
    dir_name='original',
    name_suffix='val',
);

# Display
This code is adapted from original code on http://cocodataset.org/

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import PIL

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
annotation_path = '{}/coco/original/ann_train.json'.format(FACADE_IMAGES_DIR)
image_dir = '{}/coco/original/images/train'.format(FACADE_IMAGES_DIR)

In [ ]:
annotation_path

In [ ]:
coco=COCO(annotation_path)

In [ ]:
coco.getCatIds()

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [ ]:
img_metadata = coco.loadImgs(7)[0]
image_path = os.path.join(image_dir, img_metadata['file_name'])

I = PIL.Image.open(image_path)

In [ ]:
img_metadata

In [ ]:
plt.axis('off')
plt.imshow(I)

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
catIds = coco.getCatIds(catNms=[]);
annIds = coco.getAnnIds(imgIds=img_metadata['id'], catIds=catIds)
#print(annIds)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

## coco format - template for annotations